'c:\\Users\\tashi\\Programming\\arXiv_bot'

In [27]:
# 使用するライブラリのインポート
import arxiv
import pandas as pd
import os

def fetch_papers_from_arxiv(search_query=None, max_results=3, csv_filename = "datas/arxiv_papers.csv") -> pd.DataFrame:
    """
    arXiv APIを使用して論文を取得し、CSVファイルに保存する関数

    Parameters:
        search_query (dict{"thema": "query"}): 検索クエリ
            ex) queries = {
                    "LLM": 'cat:cs.CL OR cat:cs.AI AND "large language model"',
                    "Machine Learning": 'cat:cs.LG OR cat:stat.ML AND "machine learning"',
                    "XAI": 'cat:cs.AI OR cat:cs.LG AND "explainable AI"'
                }
        max_results (int): 取得する論文の最大数
        csv_filename (str): 保存するCSVファイル名

    Returns:
        DataFrame: 新しい論文情報を含むDataFrame
    """
    if search_query is None:
        # テーマごとのクエリ
        queries = {
            "LLM": 'cat:cs.CL OR cat:cs.AI AND "large language model"',
            "Machine Learning": 'cat:cs.LG OR cat:stat.ML AND "machine learning"',
            "XAI": 'cat:cs.AI OR cat:cs.LG AND "explainable AI"'
        }
    
    # 新しく取得した論文の情報を格納するリスト
    new_papers_list = []

    # 既存のCSVファイルを読み込む（存在すれば）
    if os.path.exists(csv_filename):
        existing_papers_df = pd.read_csv(csv_filename)
    else:
        # CSVが存在しない場合は空のDataFrameを作成
        existing_papers_df = pd.DataFrame(columns=["Title", "Authors", "Published", "URL", "Abstract", "PDF"])

    # 既存のURL一覧を取得
    existing_urls = existing_papers_df["URL"].tolist()

    # 各クエリで論文を取得し、既存のデータと比較
    for theme, query in queries.items():
        search = arxiv.Search(
            query=query,
            max_results=max_results,  # 各テーマで取得する最大論文数
            sort_by=arxiv.SortCriterion.SubmittedDate
        )

        for result in search.results():
            # 論文情報を辞書にまとめる
            paper_info = {
                "Title": result.title,
                "Theme": theme,
                "Authors": ', '.join([author.name for author in result.authors]),
                "Published": str(result.published),
                "URL": result.entry_id,
                "Abstract": result.summary,
                "PDF": result.pdf_url
            }

            # 既存の論文リストにないか、または更新されている場合はリストに追加
            if paper_info["URL"] not in existing_urls or paper_info["Published"] != existing_papers_df.loc[existing_papers_df["URL"] == paper_info["URL"], "Published"].values[0]:
                new_papers_list.append(paper_info)

    # 新しい論文があれば処理
    if new_papers_list:
        # 新しい論文のDataFrameを作成
        new_papers_df = pd.DataFrame(new_papers_list)

        # 既存の論文と新しい論文を結合
        updated_papers_df = pd.concat([existing_papers_df, new_papers_df], ignore_index=True)

        # CSVファイルに上書き保存
        updated_papers_df.to_csv(csv_filename, index=False)

        # print(f"{len(new_papers_list)} 件の新しい論文を追加しました。")

    return new_papers_df


In [28]:
# from ..libs.fetch_arxiv import fetch_papers_from_arxiv
# from libs.summarize import get_summary_by_gpt
# from libs.to_message import send_message_to_discord

# def main():
#     # arXiv APIを使用して論文を取得し、CSVファイルに保存
#     df_papers = fetch_papers_from_arxiv()

In [29]:
df_papers = fetch_papers_from_arxiv()

C:\Users\tashi\AppData\Local\Temp\ipykernel_29716\76235368.py:52: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


In [35]:
import pandas as pd
from openai import OpenAI
import pandas as pd

def get_summary_by_gpt(api_key: str, df_papers: pd.DataFrame) -> dict:
    """論文の内容を要約したものを返す

    Args:
        api_key (str): OpenAI API キー
        df_papers (pd.DataFrame): 論文データ

    Returns:
        dict: レスポンスデータ（jsonをdictに変換したもの）
    """

    client = OpenAI(api_key=api_key)

    # リクエストのプロンプトを作成する
    prompt = "論文の概要を日本語でそれぞれ要約して。要約文のみ出力。"
    prompt += "出力例：pythonのlist形式。"
    prompt += str(df_papers["Abstract"].to_list())

    # OpenAI API にリクエストを送信する
    completion = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[
            {"role": "system", "content": "読者の興味を引く要素を強調"}, # 何かを知るにはまず興味をそそられることが重要
            {"role": "user", "content": prompt}
        ]
    )

    response = completion.choices[0].message.to_dict()

    return response

In [36]:
import json
key = json.load(open("openai_apikey.json"))["KEY"]

res = get_summary_by_gpt(api_key=key, df_papers=df_papers)

In [37]:
# ここにSlackなどのメッセージ送信用の関数を記述する

from discord_webhook import DiscordWebhook

def send_message_to_discord(message: str, webhook_url: str):
    """Discordにメッセージを送信する

    Args:
        message (str): 送信するメッセージ
        webhook_url (str): DiscordのWebhook URL

    Returns:
        response: webhookによるレスポンス
    """
    # メッセージを送信
    webhook = DiscordWebhook(url=webhook_url, content=message)
    response = webhook.execute()

    return response

In [48]:
summary_text = res["content"]
summary_text = summary_text[summary_text.index("["):summary_text.index("]")+1]

'[\n    "Chain-of-thought (CoT)を用いたプロンプティングは、大規模言語モデル(LLM)から推論能力を引き出すための標準的手法であるが、この「思考」が本当に有益なタスクは何かを分析した。100以上の論文に基づく定量的メタ分析を行い、14モデルにおける20のデータセットの評価を実施した結果、CoTは数学や論理に関するタスクにおいて強い性能向上を示し、他のタスクでは小幅な改善が見られた。特に、MMLUにおいては、CoTを使用しなくても答えを直接生成した場合の精度がほぼ同等であり、記号操作や推論を含む場合に限り、CoTの効果が現れた。これを受けて、計画と実行を分け、ツール拡張LLMとの比較を行った。CoTの利点の多くは記号的実行の向上から来ているが、記号ソルバーを使用するよりも劣っていることが明らかになった。CoTは選択的に適用することができ、パフォーマンスを維持しながら推論コストを節約できる。また、プロンプトベースのCoTを超え、LLMアプリケーション全体における中間計算をより活用する新しいパラダイムへの移行が必要であることを示唆している。",\n    \n    "パーソナライズされた服装推薦は、ファッションの互換性理解とトレンドの把握を必要とする複雑な課題である。本論文では、大規模言語モデル(LLM)の表現力を活かした新しいフレームワークを提案し、黒箱的性質と静的性質を微調整と直接フィードバックの統合を通じて緩和する。LLMは、人がキュレーションしたファッション画像からスタイルや色彩特性を抽出し、パーソナライズされた推薦の基盤を形成するために、画像キャプショニングを用いてアイテムの視覚的・言語的ギャップを橋渡しする。このフレームワークは、ポリヴォアデータセットで評価され、空白埋めや補完アイテムの取得において効果的であることが示された。提案したフレームワークは、基本のLLMを大幅に上回る結果を示し、シーズントレンドに沿った魅力的な提案を生成する可能性を強調している。",\n    \n    "ビッグデータと大規模言語モデルの時代が到来し、ゼロショットパーソナライズの迅速なカスタマイズが重要なトレンドとして浮上した。本報告では、特にオーディオブックの制作に特化したTakin AudioLLMという一連の技術とモデルを紹介する。このモデルは

In [52]:
from ast import literal_eval

webhookurl = json.load(open("sample_code\webhook_url.json"))["URL"]

summary_text = res["content"]
summary_text = summary_text[summary_text.index("["):summary_text.index("]")+1]
summary_list = literal_eval(summary_text)
df_papers["Summary"] = summary_list
df_papers

,Title,Theme,Authors,Published,URL,Abstract,PDF,Summary
0,To CoT or not to CoT? Chain-of-thought helps m...,LLM,"Zayne Sprague, Fangcong Yin, Juan Diego Rodrig...",2024-09-18 17:55:00+00:00,http://arxiv.org/abs/2409.12183v1,Chain-of-thought (CoT) via prompting is the de...,http://arxiv.org/pdf/2409.12183v1,Chain-of-thought (CoT)を用いたプロンプティングは、大規模言語モデル(L...
1,Decoding Style: Efficient Fine-Tuning of LLMs ...,LLM,"Najmeh Forouzandehmehr, Nima Farrokhsiar, Rami...",2024-09-18 17:15:06+00:00,http://arxiv.org/abs/2409.12150v1,Personalized outfit recommendation remains a c...,http://arxiv.org/pdf/2409.12150v1,パーソナライズされた服装推薦は、ファッションの互換性理解とトレンドの把握を必要とする複雑な課...
2,Takin: A Cohort of Superior Quality Zero-shot ...,LLM,"EverestAI, :, Sijin Chen, Yuan Feng, Laipeng H...",2024-09-18 17:03:12+00:00,http://arxiv.org/abs/2409.12139v1,With the advent of the big data and large lang...,http://arxiv.org/pdf/2409.12139v1,ビッグデータと大規模言語モデルの時代が到来し、ゼロショットパーソナライズの迅速なカスタマイズ...
3,Fitting Multilevel Factor Models,Machine Learning,"Tetiana Parshakova, Trevor Hastie, Stephen Boyd",2024-09-18 15:39:12+00:00,http://arxiv.org/abs/2409.12067v1,We examine a special case of the multilevel fa...,http://arxiv.org/pdf/2409.12067v1,多層ファクターモデルの特別なケースを調査し、共分散を多層低ランク(MLR)行列により定義する...
4,Cartan moving frames and the data manifolds,Machine Learning,"Eliot Tron, Rita Fioresi, Nicolas Couellan, St...",2024-09-18 15:31:29+00:00,http://arxiv.org/abs/2409.12057v1,The purpose of this paper is to employ the lan...,http://arxiv.org/pdf/2409.12057v1,本稿では、Cartan移動フレームの言語を用いてデータ多様体の幾何学とリーマン構造を研究し、...
5,Symmetry-Based Structured Matrices for Efficie...,Machine Learning,"Ashwin Samudre, Mircea Petrache, Brian D. Nord...",2024-09-18 07:52:33+00:00,http://arxiv.org/abs/2409.11772v1,There has been much recent interest in designi...,http://arxiv.org/pdf/2409.11772v1,最近、緩和可変性を持つ対称性を意識したニューラルネットワーク(NNs)の設計に多くの関心が集...
6,Towards Interpretable End-Stage Renal Disease ...,XAI,"Yubo Li, Saba Al-Sayouri, Rema Padman",2024-09-18 16:03:57+00:00,http://arxiv.org/abs/2409.12087v1,This study explores the potential of utilizing...,http://arxiv.org/pdf/2409.12087v1,この研究は、管理請求データと先進的な機械学習、ディープラーニング技術を組み合わせて、慢性腎疾...
7,MonoKAN: Certified Monotonic Kolmogorov-Arnold...,XAI,"Alejandro Polo-Molina, David Alfaya, Jose Portela",2024-09-17 11:10:59+00:00,http://arxiv.org/abs/2409.11078v1,Artificial Neural Networks (ANNs) have signifi...,http://arxiv.org/pdf/2409.11078v1,人工ニューラルネットワーク(ANN)は、パターン認識や複雑な問題解決において顕著な進展を遂げ...
8,Trustworthy Conceptual Explanations for Neural...,XAI,"Som Sagar, Aditya Taparia, Harsh Mankodiya, Pr...",2024-09-16 21:11:12+00:00,http://arxiv.org/abs/2409.10733v1,Black box neural networks are an indispensable...,http://arxiv.org/pdf/2409.10733v1,ブラックボックスニューラルネットワークは、現代のロボットに不可欠な要素である。しかし、高ステ...


In [68]:
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')

message = f"## {today} の最新論文をお送ります。"
for i, row in enumerate(df_papers.iterrows()):
    if i % 3 == 0:
        send_message_to_discord(message, webhookurl)
        message = ""
    row = row[1]
    message += f"\n### タイトル: {row['Title']}\n**検索条件: {row['Theme']}**\nURL: <{row['URL']}>\nPDF: <{row['PDF']}>\n要約: {row['Summary']}"
send_message_to_discord(message, webhookurl)

## 2024-09-20 の最新論文をお送ります。

### タイトル: To CoT or not to CoT? Chain-of-thought helps mainly on math and symbolic reasoning
**検索条件: LLM**
URL: <http://arxiv.org/abs/2409.12183v1>
PDF: <http://arxiv.org/pdf/2409.12183v1>
要約: Chain-of-thought (CoT)を用いたプロンプティングは、大規模言語モデル(LLM)から推論能力を引き出すための標準的手法であるが、この「思考」が本当に有益なタスクは何かを分析した。100以上の論文に基づく定量的メタ分析を行い、14モデルにおける20のデータセットの評価を実施した結果、CoTは数学や論理に関するタスクにおいて強い性能向上を示し、他のタスクでは小幅な改善が見られた。特に、MMLUにおいては、CoTを使用しなくても答えを直接生成した場合の精度がほぼ同等であり、記号操作や推論を含む場合に限り、CoTの効果が現れた。これを受けて、計画と実行を分け、ツール拡張LLMとの比較を行った。CoTの利点の多くは記号的実行の向上から来ているが、記号ソルバーを使用するよりも劣っていることが明らかになった。CoTは選択的に適用することができ、パフォーマンスを維持しながら推論コストを節約できる。また、プロンプトベースのCoTを超え、LLMアプリケーション全体における中間計算をより活用する新しいパラダイムへの移行が必要であることを示唆している。
### タイトル: Decoding Style: Efficient Fine-Tuning of LLMs for Image-Guided Outfit Recommendation with Preference
**検索条件: LLM**
URL: <http://arxiv.org/abs/2409.12150v1>
PDF: <http://arxiv.org/pdf/2409.12150v1>
要約: パーソナライズされた服装推薦は、ファッションの互換性理解とトレンドの把握を必要とする複雑な課題である。本論文では、大規模言語モデル(LLM)の表現力を活かした新しいフレームワ

In [67]:
df_papers

,Title,Theme,Authors,Published,URL,Abstract,PDF,Summary
0,To CoT or not to CoT? Chain-of-thought helps m...,LLM,"Zayne Sprague, Fangcong Yin, Juan Diego Rodrig...",2024-09-18 17:55:00+00:00,http://arxiv.org/abs/2409.12183v1,Chain-of-thought (CoT) via prompting is the de...,http://arxiv.org/pdf/2409.12183v1,Chain-of-thought (CoT)を用いたプロンプティングは、大規模言語モデル(L...
1,Decoding Style: Efficient Fine-Tuning of LLMs ...,LLM,"Najmeh Forouzandehmehr, Nima Farrokhsiar, Rami...",2024-09-18 17:15:06+00:00,http://arxiv.org/abs/2409.12150v1,Personalized outfit recommendation remains a c...,http://arxiv.org/pdf/2409.12150v1,パーソナライズされた服装推薦は、ファッションの互換性理解とトレンドの把握を必要とする複雑な課...
2,Takin: A Cohort of Superior Quality Zero-shot ...,LLM,"EverestAI, :, Sijin Chen, Yuan Feng, Laipeng H...",2024-09-18 17:03:12+00:00,http://arxiv.org/abs/2409.12139v1,With the advent of the big data and large lang...,http://arxiv.org/pdf/2409.12139v1,ビッグデータと大規模言語モデルの時代が到来し、ゼロショットパーソナライズの迅速なカスタマイズ...
3,Fitting Multilevel Factor Models,Machine Learning,"Tetiana Parshakova, Trevor Hastie, Stephen Boyd",2024-09-18 15:39:12+00:00,http://arxiv.org/abs/2409.12067v1,We examine a special case of the multilevel fa...,http://arxiv.org/pdf/2409.12067v1,多層ファクターモデルの特別なケースを調査し、共分散を多層低ランク(MLR)行列により定義する...
4,Cartan moving frames and the data manifolds,Machine Learning,"Eliot Tron, Rita Fioresi, Nicolas Couellan, St...",2024-09-18 15:31:29+00:00,http://arxiv.org/abs/2409.12057v1,The purpose of this paper is to employ the lan...,http://arxiv.org/pdf/2409.12057v1,本稿では、Cartan移動フレームの言語を用いてデータ多様体の幾何学とリーマン構造を研究し、...
5,Symmetry-Based Structured Matrices for Efficie...,Machine Learning,"Ashwin Samudre, Mircea Petrache, Brian D. Nord...",2024-09-18 07:52:33+00:00,http://arxiv.org/abs/2409.11772v1,There has been much recent interest in designi...,http://arxiv.org/pdf/2409.11772v1,最近、緩和可変性を持つ対称性を意識したニューラルネットワーク(NNs)の設計に多くの関心が集...
6,Towards Interpretable End-Stage Renal Disease ...,XAI,"Yubo Li, Saba Al-Sayouri, Rema Padman",2024-09-18 16:03:57+00:00,http://arxiv.org/abs/2409.12087v1,This study explores the potential of utilizing...,http://arxiv.org/pdf/2409.12087v1,この研究は、管理請求データと先進的な機械学習、ディープラーニング技術を組み合わせて、慢性腎疾...
7,MonoKAN: Certified Monotonic Kolmogorov-Arnold...,XAI,"Alejandro Polo-Molina, David Alfaya, Jose Portela",2024-09-17 11:10:59+00:00,http://arxiv.org/abs/2409.11078v1,Artificial Neural Networks (ANNs) have signifi...,http://arxiv.org/pdf/2409.11078v1,人工ニューラルネットワーク(ANN)は、パターン認識や複雑な問題解決において顕著な進展を遂げ...
8,Trustworthy Conceptual Explanations for Neural...,XAI,"Som Sagar, Aditya Taparia, Harsh Mankodiya, Pr...",2024-09-16 21:11:12+00:00,http://arxiv.org/abs/2409.10733v1,Black box neural networks are an indispensable...,http://arxiv.org/pdf/2409.10733v1,ブラックボックスニューラルネットワークは、現代のロボットに不可欠な要素である。しかし、高ステ...
